# Import and Loading Libraries

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import module, secret_keys
from model_list import models
import pandas as pd

hf_api_key             = secret_keys.HF_TOKEN                   #<insert your own huggingface token here>
openai_api_key         = secret_keys.OPENAI_API_KEY_TEAM        #<insert your own openai token here>

# Load Dataset

In [4]:
data = pd.read_csv('data/Gold-100.csv')
print(data.shape)
data.head(2)

(103, 16)


,NCTId,BriefTitle,EligibilityCriteria,BriefSummary,Conditions,StudyType,Interventions,PrimaryOutcomes,BaselineMeasures,Paper_BaselineMeasures,LLama3_70b_in,LLama3_70b_in_threeshot,GPT4_Turbo_in,GPT4_Turbo_in_threeshot,GPT4_Omni_in,GPT4_Omni_in_threeshot
0,NCT02572882,Gut Microbiome and p-Inulin in Hemodialysis,Inclusion Criteria:\n\n* Maintenance hemodialy...,"The Microbiome trial is a non-randomized, open...","End-Stage Renal Disease, Gut Microbiome Dysbio...",INTERVENTIONAL,"p-inulin,",Within Participant Variability in Microbiome C...,"Age, Categorical, Age, Continuous, Sex: Female...","Age, Sex, Race, Ethnicity, Hypertension, Diabe...","Age, sex, race, ethnicity, body mass index, he...","Age, Sex, Ethnicity, Body Mass Index (BMI), He...","Age, Gender, Duration of hemodialysis therapy,...","Age, Sex, Duration of hemodialysis, Frequency ...","Age, gender, duration of hemodialysis, stool f...","Age, Sex, Duration of hemodialysis, Stool freq..."
1,NCT02623348,Use of Pedometers to Measure and Increase Walk...,Inclusion Criteria:\n\n* on hemodialysis for ≥...,Randomized controlled trial using pedometers t...,"End-stage Renal Disease,",INTERVENTIONAL,"pedometer,","Physical Activity,","Age, Continuous, Sex: Female, Male, Ethnicity ...","Age, Sex, Race, Ethnicity, BMI, HTN, DM, CAD, ...","age, sex, body mass index, dialysis duration, ...","Age, Sex, Ethnicity, BMI, Dialysis vintage, Co...","age, gender, duration on hemodialysis, baselin...","Age, Sex, Duration on dialysis, Baseline physi...","Age, gender, duration on hemodialysis, baselin...","Age, Sex, Duration of hemodialysis, Baseline p..."


## Example of Generation using GPT-4o with 0-shot prompts

In [7]:
model_name   = models['gpt-4o']

row = next(data.iterrows())[1]
system_message, question = module.build_zeroshot_prompt(data, row)
model_query = module.system_user_template(system_message, question)
model_response = module.run_generation_single_openai(model_query, model_name, 
                                                openai_api_key, temperature=0.0)
print(system_message)
print(question)
print(model_response)


You are a helpful assistant with experience in the clinical domain and clinical trial design.     You'll be asked queries related to clinical trials. These inquiries will be delineated by a '##Question' heading.     Inside these queries, expect to find comprehensive details about the clinical trial structured within specific subsections,     indicated by '<>' tags. These subsections include essential information such as the trial's title, brief summary,     condition under study, inclusion and exclusion criteria, intervention, and outcomes.In answer to this question, return a list of probable baseline features (separated by commas) of the clinical trial.     Baseline features are the set of baseline or demographic characteristics that are assessed at baseline and used in the analysis of the     primary outcome measure(s) to characterize the study population and assess validity. Clinical trial-related publications typically     include a table of baseline features assessed  by arm or co

## Example of Generation using llama3-70b-it with 3-shot prompts

In [6]:
model_hf_endpoint = models['llama3-70b-it']

row = next(data.iterrows())[1]
system_message, question = module.build_three_shot_prompt(data, row)
model_query = module.system_user_template(system_message, question)
model_response = module.run_generation_single_hf_models(model_query, model_hf_endpoint, 
                                                hf_api_key, temperature=0.0)
print(system_message)
print(question)
print(model_response)


You are a helpful assistant with experience in the clinical domain and clinical trial design.     You'll be asked queries related to clinical trials. These inquiries will be delineated by a '##Question' heading.     Inside these queries, expect to find comprehensive details about the clinical trial structured within specific subsections,     indicated by '<>' tags. These subsections include essential information such as the trial's title, brief summary,     condition under study, inclusion and exclusion criteria, intervention, and outcomes.In answer to this question, return a list of probable baseline features (separated by commas) of the clinical trial.     Baseline features are the set of baseline or demographic characteristics that are assessed at baseline and used in the analysis of the     primary outcome measure(s) to characterize the study population and assess validity. Clinical trial-related publications typically     include a table of baseline features assessed  by arm or co

# Example of Eval using GPT-4o

In [9]:
row = next(data.iterrows())[1]
reference_list = row['Paper_BaselineMeasures']
candidate_list = row['LLama3_70b_in']

qstart = module.get_question_from_row(row)
system_message, question = module.build_gpt4_eval_prompt(reference_list, candidate_list, qstart)
model_response = module.run_evaluation_with_gpt4o(system_message, question, openai_api_key)

print(system_message)
print(question)
print(model_response)

TypeError: run_evaluation_with_gpt4o() missing 1 required positional argument: 'openai_token'